<a href="https://colab.research.google.com/github/DGkurnia/cnkproject/blob/main/BMLP_Derfansyah_Guswiranata_Kurnia_klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
#Perpustakaan untuk pengelolaan data
import numpy as np
import pandas as pd
#Grafik
import matplotlib.pyplot as plt #grafik dasar
import seaborn as sns
import datetime

In [6]:
#pelabelan untuk 'onehotencoder'
from sklearn.preprocessing import OneHotEncoder as ohen

In [29]:
#Bagian svm(p1)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import RobustScaler, MinMaxScaler

In [ ]:
#Bagian klasifikasi 'randomforest'
from sklearn.ensemble import RandomForestClassifier

In [10]:
#Pengunduhan dataset ke google colab
from google.colab import files
#ke proses 'upload'

In [11]:
#proses 'upload'
uploaded = files.upload()

Saving hslclstr.csv to hslclstr.csv


In [8]:
#pengelolaan file
import os #untuk os
import time #waktu
import datetime #tanggal
import gc #pengunduhan file per batch

In [9]:
#Penyimpanan hasil sementara di google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
#Bagian multiprocessing melalui instalasi pool
import warnings #peringatan
import multiprocessing as mp #untuk multiprocessing
from multiprocessing import Pool

In [14]:
#penyimpanan hasil melalui dill
! pip install dill
import dill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.0 MB/s eta 0:00:00


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [15]:
#Cek data
df = pd.read_csv('/content/hslclstr.csv')
dfc = df.copy() #jangan pakai data asli

In [16]:
#cek hasil
smpl = dfc.head(7)
inf = dfc.info() #informasi

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   img                          64016 non-null  object 
 1   title                        64016 non-null  object 
 2   console                      64016 non-null  object 
 3   genre                        64016 non-null  object 
 4   publisher                    64016 non-null  object 
 5   total_sales                  64016 non-null  float64
 6   na_sales                     64016 non-null  float64
 7   jp_sales                     64016 non-null  float64
 8   pal_sales                    64016 non-null  float64
 9   other_sales                  64016 non-null  float64
 10  critic_score                 64016 non-null  float64
 11  release_date                 64016 non-null  object 
 12  last_update                  64016 non-null  object 
 13  kemungkinan gene

In [18]:
#inspeksi label untuk sampel
lbl = dfc['Label'].copy()
print(lbl.head(7))

0    game kurang berkenan dengan fokus pencarian ke...
1    game kurang berkenan dengan fokus pencarian ke...
2    game kurang berkenan dengan fokus pencarian ke...
3                game dengan tujuan utama pencari duit
4    game kurang berkenan dengan fokus pencarian ke...
5                game dengan tujuan utama pencari duit
6                game dengan tujuan utama pencari duit
Name: Label, dtype: object


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [20]:
#Inisiasi pembagian data
inspk = dfc.iloc[:,5:11].copy() #numerikal untuk klasifikasi kluster
inspk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   total_sales   64016 non-null  float64
 1   na_sales      64016 non-null  float64
 2   jp_sales      64016 non-null  float64
 3   pal_sales     64016 non-null  float64
 4   other_sales   64016 non-null  float64
 5   critic_score  64016 non-null  float64
dtypes: float64(6)
memory usage: 2.9 MB


In [21]:
#numerisasi label (identifikasi jenis label)
jnslbl = dfc['Label'].copy()
jnslbl.info()

<class 'pandas.core.series.Series'>
RangeIndex: 64016 entries, 0 to 64015
Series name: Label
Non-Null Count  Dtype 
--------------  ----- 
64016 non-null  object
dtypes: object(1)
memory usage: 500.2+ KB


In [26]:
#Sumbu x
xcls = inspk.copy() #pakai numerik

Sumbu x  **(xcls)** adalah nilai numerikal dari tabel yang terdiri dari
* Penjualan dari lima indikator (total, spesifika amerika utara, jepang, PAL, lain-lain)
* Nilai kritik

In [23]:
#Persiapan pembagian data (untuk kluster)
ycls = jnslbl #masih kata-kata
#pelabelan dengan 'ohen.onehotencoder' (pembentukan ulang)
bntkbaru, lbl = pd.factorize(ycls)
#inspeksi hasil
print(bntkbaru)
print(lbl)

[0 0 0 ... 2 2 2]
Index(['game kurang berkenan dengan fokus pencarian keuntungan',
       'game dengan tujuan utama pencari duit',
       'game favortit dengan keuntungan rendah'],
      dtype='object')


sampel kolom pelabelan

In [24]:
#pembentukan ulang
enkoding = bntkbaru.reshape(-1,1)
#inisiasi ohen
encoder = ohen()
#penyesuaian data
lblohen = encoder.fit_transform(enkoding)

In [25]:
#konversi ke df untuk kluster jadi
dflbl = pd.DataFrame(lblohen.toarray(), columns=lbl)
smpllbl = dflbl.head(7)
print(smpllbl) #inspeksi hasil

   game kurang berkenan dengan fokus pencarian keuntungan  \
0                                                1.0        
1                                                1.0        
2                                                1.0        
3                                                0.0        
4                                                1.0        
5                                                0.0        
6                                                0.0        

   game dengan tujuan utama pencari duit  \
0                                    0.0   
1                                    0.0   
2                                    0.0   
3                                    1.0   
4                                    0.0   
5                                    1.0   
6                                    1.0   

   game favortit dengan keuntungan rendah  
0                                     0.0  
1                                     0.0  
2                         

Nilai ini **(ycls)** menjadi komponen untuk penyatuan

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [32]:
#proses scaling untuk sumbu X
rbsscal = RobustScaler() #untuk SVM
xclscl = rbsscal.fit_transform(xcls) #untuk metode SVM
#pengubahan ke data frame untuk evaluasi
dfxcl = pd.DataFrame(xclscl, columns=xcls.columns)

In [33]:
#inspeksi kolom
print(dfxcl.head(7))

   total_sales  na_sales  jp_sales  pal_sales  other_sales  critic_score
0       1016.0     637.0      99.0       9.85         3.12      1.105263
1        969.5     606.0      60.0       9.71         3.02      1.263158
2        807.5     841.0      47.0       5.49         1.78      1.210526
3        793.0     906.0       6.0       5.33         1.42      1.210526
4        754.5     618.0      41.0       6.05         2.44      0.421053
5        741.0     907.0      13.0       4.29         1.33      0.736842
6        737.0     976.0      11.0       3.73         1.14      0.789474


7 kolom

In [30]:
#Persiapan df ternormalisasi untuk Random forest classifier
mnmxscal = MinMaxScaler() #untuk RCT ('RandomforestClasTree')
xclsmm = mnmxscal.fit_transform(xcls)

In [34]:
#Bentuk nilai data untuk teknik RCT
dfxclmm = pd.DataFrame(xclsmm, columns=xcls.columns)
#cek hasil
print(dfxclmm.head(7))

   total_sales  na_sales  jp_sales  pal_sales  other_sales  critic_score
0     1.000000  0.652664  0.464789   1.000000     1.000000      0.933333
1     0.954232  0.620902  0.281690   0.985787     0.967949      0.966667
2     0.794783  0.861680  0.220657   0.557360     0.570513      0.955556
3     0.780512  0.928279  0.028169   0.541117     0.455128      0.955556
4     0.742618  0.633197  0.192488   0.614213     0.782051      0.788889
5     0.729331  0.929303  0.061033   0.435533     0.426282      0.855556
6     0.725394  1.000000  0.051643   0.378680     0.365385      0.866667


In [39]:
#deklarasi nilai X dan y
X_train, X_test, y_train, y_test = train_test_split(xclscl, ycls, test_size=0.225, random_state=42) #untuk SVM
X_train2, X_test2, y_train2, y_test2 = train_test_split(xclscl, ycls, test_size=0.225, random_state=42) #untuk RCT

In [36]:
#Fungsi penentu akurasi di SVM
def train_and_evaluate(params):
    C, gamma = params
    model = SVC(C=C, gamma=gamma)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Inspeksi indikator tesebut
    if (accuracy >= 0.87 and precision >= 0.87 and recall >= 0.87 and f1 >= 0.87) or \
       (accuracy >= 0.92 and precision >= 0.92 and recall >= 0.92 and f1 >= 0.92):
        return (C, gamma, accuracy, precision, recall, f1, y_pred)
    elif (accuracy >= 0.87 and precision >= 0.87 and recall >= 0.87 and f1 >= 0.87) or \
       (accuracy < 0.92 and precision < 0.92 and recall < 0.92 and f1 < 0.92):
       print("Hasil masih layak untuk disimpan.")
       return (C, gamma, accuracy, precision, recall, f1, y_pred)
    else:
       print("Hasil tidak layak untuk disimpan.")
       return None
  #ke paralelisasi melalui SVM

In [37]:
#Fungsi evaluasi matriks
def generate_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    cm_rounded = np.round(cm.astype(float), decimals=4)  # Pembulatan untuk kemudahan nilai
    print("Confusion Matrix:")
    print(cm_rounded)
    return cm_rounded

In [ ]:
#Persiapan fungsi Randomforestclassifier (fungsi pelatihan)
def latihan_rfc (X_train, y_train):
    # Pelacakan estimator dan
    model = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=42)

    # Define hyperparameters to tune (you can modify these)
    param_grid = {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }

    # Use GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    return grid_search.best_estimator_
#ke paralelisasi

In [ ]:
#Paralelisasi
def parallel_training(X_train, y_train):
    with mp.Pool(processes=mp.cpu_count()) as pool:
        model = pool.apply(latihan_rfc, (X_train, y_train))
    return model
#ke evaluasi

In [ ]:
#Evaluasi teknik RCT
def evaluasi(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

Tulis narasi atau penjelasan algoritma yang Anda gunakan.
### Rincian pemakaian algoritma SVM (Support vector machines)
1. Persiapan numerisasi label **'onehotencoding'**
2. Proses standardisasi kolom numerik dengan **"Robustscaler"**
3. Eksekusi dengan teknik SVM melalui beberapa tahap :
#### 3. a. Proses Pelatihan dan evaluasi nilai gama dalam jangkauan dinamis
#### 3. b. Proses Paralelisasi untuk eksekusi
### Rincian pemkaian algoritma RFC (Random Forest Classifier)
* Persiapan numerisasi label dengan **'onehotencoding'**
* proses standardisasi dengan metode sama (agar lebih adil)
* Eksekusi fungsi yang terdiri dari
** a. Pelatihan RFC dengan GridSearchCV (akan diganti dengan randomsearch di modifikasi nanti)
** b. Tahapan paralelisasi

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [ ]:
#Paralelisasi melalui SVM
if __name__ == '__main__':
    # Define a parameter grid with a range of gamma values
    param_grid = [(C, gamma) for C in [0.1, 1, 10] for gamma in np.linspace(0.01, 1, num=10)]

    with Pool() as pool:
        results = pool.map(train_and_evaluate, param_grid)

    # Penyaringan hasil gagal
    filtered_results = [result for result in results if result is not None]

    # Save results using dill
    with open('svm_results.dill', 'wb') as f:
        dill.dump(filtered_results, f)

    # Penghasilan data
    for result in filtered_results:
        _, _, _, _, _, _, y_pred = result
        cm = generate_confusion_matrix(y_test, y_pred)

        # Optionally save each confusion matrix using dill
        with open(f'confusion_matrix_C_{result[0]}_gamma_{result[1]}.dill', 'wb') as cm_file:
            dill.dump(cm, cm_file)

    # Filtrasi Hasil
    for result in filtered_results:
        print(result)

In [ ]:
#eksekusi dengan RCT
model = parallel_training(X_train2, y_train2)
accuracy, precision, recall, f1 = evaluasi(model, X_test, y_test)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## Evaluasi algoritma

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

## Gridsearch di teknik SVM

In [ ]:
#Inisiasi grid search di SVM

In [ ]:
#Lanjutan di SVM


## Gridsearch di RCT

In [ ]:
#Tahap awal di RCT


In [ ]:
#Lanjutan di RCT

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [ ]:
#Inspeksi di metode Support Vector Machine


In [ ]:
#Inspeksi di metode RandomForestClasifier


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.